In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import optuna
import lightgbm
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score

import os
import yaml
import sys

sys.path.append('../src')
import utils.utils as utils

!pip freeze > ../requirements.txt

d:\huynq1\202507-homecredit-pd\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_set = pd.read_csv('../data/cleaned_train_data.csv')
test_set = pd.read_csv('../data/cleaned_test_data.csv')

In [5]:
X_train = train_set.drop(['SK_ID_CURR', 'TARGET'], axis=1)
y_train = train_set.TARGET
X_test = test_set.drop(['SK_ID_CURR', 'TARGET'], axis=1)
y_test = test_set.TARGET

# Double check categorical cols
cat_cols = [col for col in X_train.columns if X_train[col].dtype == 'object' or X_train[col].dtype == 'category']
for col in cat_cols:
    X_train[col] = X_train[col].astype('category')
    X_test[col] = X_test[col].astype('category')

In [8]:
def objective(trial):
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 5.0),
        'verbosity': -1,
        'random_state': 42
    }

    # K-Fold
    kf = KFold(n_splits=5, shuffle=True, random_state=810)
    auc_scores = []

    for train_idx, val_idx in kf.split(X_train, y_train):
        X_train_kf, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_train_kf, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        train_data = lightgbm.Dataset(X_train_kf, label=y_train_kf, categorical_feature=cat_cols)
        val_data = lightgbm.Dataset(X_val, label=y_val, reference=train_data)

        model = lightgbm.train(
            params,
            train_data,
            valid_sets=[val_data],
            num_boost_round=1000,
            callbacks=[
                lightgbm.early_stopping(stopping_rounds=50)
            ]
        )

        preds = model.predict(X_val)
        auc = roc_auc_score(y_val, preds)
        auc_scores.append(auc)
    return np.mean(auc_scores)

In [11]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2025-07-22 12:30:09,825] A new study created in memory with name: no-name-df1889d9-0367-4ff9-a48d-5d3d94dadd1e


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's auc: 0.767964
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's auc: 0.765636
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's auc: 0.777897
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's auc: 0.769222
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's auc: 0.771336


[I 2025-07-22 12:36:16,277] Trial 0 finished with value: 0.7704108502941287 and parameters: {'learning_rate': 0.010700081323484994, 'num_leaves': 25, 'max_depth': 12, 'min_child_samples': 55, 'subsample': 0.6647134992996024, 'colsample_bytree': 0.6830899163903577, 'reg_alpha': 4.783980657371293, 'reg_lambda': 1.4082825060734694}. Best is trial 0 with value: 0.7704108502941287.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[51]	valid_0's auc: 0.763527
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[69]	valid_0's auc: 0.760742
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[58]	valid_0's auc: 0.771928
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[56]	valid_0's auc: 0.762489
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[57]	valid_0's auc: 0.765152


[I 2025-07-22 12:37:15,973] Trial 1 finished with value: 0.7647676096805673 and parameters: {'learning_rate': 0.20685533836029116, 'num_leaves': 62, 'max_depth': 10, 'min_child_samples': 84, 'subsample': 0.7717793136210099, 'colsample_bytree': 0.7191099876905531, 'reg_alpha': 4.808599818270697, 'reg_lambda': 4.376241480157995}. Best is trial 0 with value: 0.7704108502941287.


...

[I 2025-07-22 13:22:57,542] Trial 47 finished with value: 0.7704163603988967 and parameters: {'learning_rate': 0.07137603539815429, 'num_leaves': 68, 'max_depth': 7, 'min_child_samples': 35, 'subsample': 0.8047506548594076, 'colsample_bytree': 0.8279226923661777, 'reg_alpha': 1.768036706275858, 'reg_lambda': 2.139415381163314}. Best is trial 45 with value: 0.7722052869742261.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[939]	valid_0's auc: 0.770196
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[914]	valid_0's auc: 0.767039
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[603]	valid_0's auc: 0.779067
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[908]	valid_0's auc: 0.77087
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[994]	valid_0's auc: 0.772777


[I 2025-07-22 13:25:10,885] Trial 48 finished with value: 0.7719896495003711 and parameters: {'learning_rate': 0.02382466367967133, 'num_leaves': 66, 'max_depth': 6, 'min_child_samples': 26, 'subsample': 0.7655531176888467, 'colsample_bytree': 0.5535828073978959, 'reg_alpha': 1.9610984091385195, 'reg_lambda': 1.6089794045561046}. Best is trial 45 with value: 0.7722052869742261.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[918]	valid_0's auc: 0.770439
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's auc: 0.768299
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[976]	valid_0's auc: 0.780206
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[755]	valid_0's auc: 0.770638
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[904]	valid_0's auc: 0.7726


[I 2025-07-22 13:28:01,620] Trial 49 finished with value: 0.772436315663825 and parameters: {'learning_rate': 0.02280647543938272, 'num_leaves': 64, 'max_depth': 6, 'min_child_samples': 17, 'subsample': 0.7647469315402096, 'colsample_bytree': 0.5673960159030922, 'reg_alpha': 3.441721376193107, 'reg_lambda': 1.710834070720233}. Best is trial 49 with value: 0.772436315663825.


In [15]:
print(f'Best AUC: {study.best_value:.4f}')
print(study.best_params)
config_path = '../configs/model_config.yaml'
with open(config_path, 'w') as file:
    yaml.dump(study.best_params, file, default_flow_style=False)
print('Dumped params')

Best AUC: 0.7724
{'learning_rate': 0.02280647543938272, 'num_leaves': 64, 'max_depth': 6, 'min_child_samples': 17, 'subsample': 0.7647469315402096, 'colsample_bytree': 0.5673960159030922, 'reg_alpha': 3.441721376193107, 'reg_lambda': 1.710834070720233}
Dumped params
